In [ ]:
# Cell 1: Mount Google Drive and set up directories
from google.colab import drive
drive.mount('/content/drive')

# Create directories in Drive
!mkdir -p /content/drive/MyDrive/email_bert
!mkdir -p /content/drive/MyDrive/email_bert/datasets
!mkdir -p /content/drive/MyDrive/email_bert/data
!mkdir -p /content/drive/MyDrive/email_bert/tokenizer
!mkdir -p /content/drive/MyDrive/email_bert/checkpoints

# Install required packages
!pip install transformers datasets tokenizers pandas email-parser

# Download Enron Email Dataset (smaller subset for pre-training)
!wget https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz


Mounted at /content/drive
ERROR: Could not find a version that satisfies the requirement email-parser (from versions: none)
ERROR: No matching distribution found for email-parser
--2025-10-21 00:30:16--  https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz [following]
--2025-10-21 00:30:17--  https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz
Reusing existing connection to www.cs.cmu.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: 443254787 (423M) [application/x-gzip]
Saving to: ‘enron_mail_20150507.tar.gz’

enron_mail_20150507 100%[===================>] 422.72M  2.13MB/s    in 3m 33s  

2025-10-21 00:33:50 (1.98 MB/s) - ‘enron_mail_20150507.tar.gz’ saved [443254787/443254787]

Processing Enron emails...


FileNotFoundError: [Errno 2] No such file or directory: './enron_mail_20150507/maildir/'

In [ ]:
# First, ensure drive is mounted
from google.colab import drive
drive.mount('/content/drive')

# Create the destination directory
!mkdir -p /content/drive/MyDrive/email_bert/datasets/

# Download directly to Drive
!wget -O /content/drive/MyDrive/email_bert/datasets/enron_mail_20150507.tar.gz https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz

# Change to the Drive directory
%cd /content/drive/MyDrive/email_bert/datasets/

# Extract the archive directly in Drive
!tar -xzf enron_mail_20150507.tar.gz

# Update the path in your code
enron_dir = '/content/drive/MyDrive/email_bert/datasets/maildir/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--2025-10-21 00:49:57--  https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz [following]
--2025-10-21 00:49:57--  https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz
Reusing existing connection to www.cs.cmu.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: 443254787 (423M) [application/x-gzip]
Saving to: ‘/content/drive/MyDrive/email_bert/datasets/enron_mail_20150507.tar.gz’

/content/drive/MyDr 100%[===================>] 422.72M  1.08MB/s    in 4m 16s  

2025-10-21 00:54:13 (1.65 MB/s) - ‘/content/drive/MyDrive/email_bert/datasets/enron_mail_20150507.tar.gz’ saved [443254787/443254787]

In [9]:
# Extract the archive to the local Colab VM storage first
!tar -xzf /content/drive/MyDrive/email_bert/datasets/enron_mail_20150507.tar.gz -C /content/

# Then we'll work with the locally extracted files


# Now we can continue with the email processing
# Change your code to use this path

In [11]:

# Extract the downloaded tarball
enron_dir = '/content/maildir/'

# Data processing: Process email data instead of movie dialogues
import os
import email
import pandas as pd
from pathlib import Path
import torch
import re
import random
import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam
from email.parser import Parser
import json

def extract_email_content(email_path):
    """Extract content from an email file"""
    try:
        with open(email_path, 'r', encoding='latin1') as f:
            content = f.read()

        # Parse email content
        msg = email.message_from_string(content)

        # Get subject and body
        subject = msg.get('Subject', '')

        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                ctype = part.get_content_type()
                cdispo = str(part.get('Content-Disposition'))

                # Skip attachments
                if ctype == 'text/plain' and 'attachment' not in cdispo:
                    part_body = part.get_payload(decode=True)
                    if part_body:
                        body = part_body.decode('latin1', errors='ignore')
                    break
        else:
            part_body = msg.get_payload(decode=True)
            if part_body:
                body = part_body.decode('latin1', errors='ignore')

        # Clean text - remove excessive whitespace
        body = re.sub(r'\s+', ' ', body).strip() if body else ""
        subject = re.sub(r'\s+', ' ', subject).strip()

        return subject, body
    except Exception as e:
        return "", ""

# Process Enron emails
print("Processing Enron emails...")
MAX_LEN = 64
#enron_dir = '/content/drive/MyDrive/email_bert/datasets/enron_mail_20150507/maildir/'
email_pairs = []

# Limit to a sample to avoid processing too many emails
users = os.listdir(enron_dir)[:30]  # Take first 30 users

for user in users:
    user_dir = os.path.join(enron_dir, user)
    if os.path.isdir(user_dir):
        for folder in os.listdir(user_dir)[:5]:  # Limit folders per user
            folder_path = os.path.join(user_dir, folder)
            if os.path.isdir(folder_path):
                for file in os.listdir(folder_path)[:50]:  # Limit files per folder
                    file_path = os.path.join(folder_path, file)
                    if os.path.isfile(file_path):
                        subject, body = extract_email_content(file_path)

                        # Skip empty emails
                        if not body:
                            continue

                        # Create sentence pairs for NSP
                        # 1. Split email body into paragraphs
                        paragraphs = body.split('\n')
                        paragraphs = [p.strip() for p in paragraphs if p.strip()]

                        if len(paragraphs) < 2:
                            # If no clear paragraphs, split by sentences
                            sentences = re.split(r'[.!?]+', body)
                            sentences = [s.strip() for s in sentences if len(s.strip()) > 20]

                            if len(sentences) >= 2:
                                # Create pairs from consecutive sentences
                                for i in range(len(sentences) - 1):
                                    # Positive example: consecutive sentences
                                    first = sentences[i]
                                    second = sentences[i + 1]

                                    if len(first.split()) > 3 and len(second.split()) > 3:
                                        email_pairs.append([
                                            ' '.join(first.split()[:MAX_LEN]),
                                            ' '.join(second.split()[:MAX_LEN])
                                        ])
                        else:
                            # Create pairs from paragraphs
                            for i in range(len(paragraphs) - 1):
                                # Positive example: consecutive paragraphs
                                first = paragraphs[i]
                                second = paragraphs[i + 1]

                                if len(first.split()) > 3 and len(second.split()) > 3:
                                    email_pairs.append([
                                        ' '.join(first.split()[:MAX_LEN]),
                                        ' '.join(second.split()[:MAX_LEN])
                                    ])

print(f"Created {len(email_pairs)} email pairs")
# Sample one pair to verify
print("Sample pair:", email_pairs[20] if len(email_pairs) > 20 else email_pairs[0])

# Save pairs to Drive for reuse
import pickle
with open('/content/drive/MyDrive/email_bert/datasets/email_pairs.pkl', 'wb') as f:
    pickle.dump(email_pairs, f)

# Save data as txt file for tokenizer training
text_data = []
file_count = 0

for sample in tqdm.tqdm([x[0] for x in email_pairs] + [x[1] for x in email_pairs]):
    text_data.append(sample)

    # Save to file once we hit the 10K mark
    if len(text_data) == 10000:
        with open(f'/content/drive/MyDrive/email_bert/data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

# Save remaining data
if text_data:
    with open(f'/content/drive/MyDrive/email_bert/data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
        fp.write('\n'.join(text_data))

paths = [str(x) for x in Path('/content/drive/MyDrive/email_bert/data').glob('**/*.txt')]
print(f"Created {len(paths)} text files for tokenizer training")

# Train tokenizer
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

tokenizer.train(
    files=paths,
    vocab_size=30_000,
    min_frequency=2,
    limit_alphabet=1000,
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
)

# Save tokenizer to Drive
tokenizer.save_model('/content/drive/MyDrive/email_bert/tokenizer', 'bert-email')
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/email_bert/tokenizer/bert-email-vocab.txt', local_files_only=True)

# Test tokenizer
sample_text = "This is an email about the project deadline tomorrow."
token_ids = tokenizer(sample_text)['input_ids']
print("Sample tokenization:", tokenizer.convert_ids_to_tokens(token_ids))

# Modify BERTDataset for email data
class BERTDataset(Dataset):
    def __init__(self, data_pair, tokenizer, seq_len=64):
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.corpus_lines = len(data_pair)
        self.lines = data_pair

    def __len__(self):
        return self.corpus_lines

    def __getitem__(self, item):
        # Step 1: get email pair, either negative or positive (for NSP)
        t1, t2, is_next_label = self.get_sent(item)

        # Step 2: replace random words with mask / random words (for MLM)
        t1_random, t1_label = self.random_word(t1)
        t2_random, t2_label = self.random_word(t2)

        # Step 3: Add special tokens (CLS, SEP)
        t1 = [self.tokenizer.vocab['[CLS]']] + t1_random + [self.tokenizer.vocab['[SEP]']]
        t2 = t2_random + [self.tokenizer.vocab['[SEP]']]
        t1_label = [self.tokenizer.vocab['[PAD]']] + t1_label + [self.tokenizer.vocab['[PAD]']]
        t2_label = t2_label + [self.tokenizer.vocab['[PAD]']]

        # Step 4: combine parts and add padding
        segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
        bert_input = (t1 + t2)[:self.seq_len]
        bert_label = (t1_label + t2_label)[:self.seq_len]
        padding = [self.tokenizer.vocab['[PAD]'] for _ in range(self.seq_len - len(bert_input))]
        bert_input.extend(padding)
        bert_label.extend(padding)
        segment_label.extend(padding)

        output = {"bert_input": bert_input,
                 "bert_label": bert_label,
                 "segment_label": segment_label,
                 "is_next": is_next_label}

        return {key: torch.tensor(value) for key, value in output.items()}

    def random_word(self, sentence):
        tokens = sentence.split()
        output_label = []
        output = []

        # 15% of the tokens would be replaced
        for i, token in enumerate(tokens):
            prob = random.random()

            # Remove cls and sep token
            token_id = self.tokenizer(token)['input_ids']
            if len(token_id) <= 2:  # Skip if only special tokens remain
                continue

            token_id = token_id[1:-1]  # Remove special tokens

            # 15% chance of altering token
            if prob < 0.15:
                prob /= 0.15

                # 80% chance change token to mask token
                if prob < 0.8:
                    for i in range(len(token_id)):
                        output.append(self.tokenizer.vocab['[MASK]'])

                # 10% chance change token to random token
                elif prob < 0.9:
                    for i in range(len(token_id)):
                        output.append(random.randrange(len(self.tokenizer.vocab)))

                # 10% chance keep token as is
                else:
                    output.extend(token_id)

                output_label.extend(token_id)

            else:
                output.extend(token_id)
                for i in range(len(token_id)):
                    output_label.append(0)  # 0 means no prediction needed

        # Flattening
        output = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output]))
        output_label = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output_label]))

        return output, output_label

    def get_sent(self, index):
        '''return email segment pair - either positive or negative example'''
        t1, t2 = self.get_corpus_line(index)

        # 50% chance for a negative sample - use random second segment
        if random.random() > 0.5:
            return t1, t2, 1  # Positive example
        else:
            return t1, self.get_random_line(), 0  # Negative example

    def get_corpus_line(self, item):
        '''return segment pair from corpus'''
        return self.lines[item][0], self.lines[item][1]

    def get_random_line(self):
        '''return random segment'''
        return self.lines[random.randrange(len(self.lines))][random.choice([0, 1])]

# Test dataset
print("Creating BERT dataset...")
train_data = BERTDataset(email_pairs, seq_len=MAX_LEN, tokenizer=tokenizer)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, pin_memory=True)
sample_data = next(iter(train_loader))
print('Batch Size', sample_data['bert_input'].size())

# Test sample data
sample_item = train_data[random.randrange(len(train_data))]
print("Sample data item:", {k: v.size() for k, v in sample_item.items()})

# The following BERT model implementation is from the original notebook
# Cell for embedding layers
class PositionalEmbedding(torch.nn.Module):
    def __init__(self, d_model, max_len=128):
        super().__init__()
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        for pos in range(max_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))

        # Register as a buffer to ensure it's moved to the right device
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # Now this will automatically be on the same device as x
        return self.pe
        # self.register_buffer('pe', pe)



class BERTEmbedding(torch.nn.Module):
    """
    BERT Embedding which is consisted with under features
        1. TokenEmbedding : normal embedding matrix
        2. PositionalEmbedding : adding positional information using sin, cos
        2. SegmentEmbedding : adding sentence segment info, (sent_A:1, sent_B:2)
        sum of all these features are output of BERTEmbedding
    """

    def __init__(self, vocab_size, embed_size, seq_len=64, dropout=0.1):
        """
        :param vocab_size: total vocab size
        :param embed_size: embedding size of token embedding
        :param dropout: dropout rate
        """

        super().__init__()
        self.embed_size = embed_size
        # (m, seq_len) --> (m, seq_len, embed_size)
        # padding_idx is not updated during training, remains as fixed pad (0)
        self.token = torch.nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.segment = torch.nn.Embedding(3, embed_size, padding_idx=0)
        self.position = PositionalEmbedding(d_model=embed_size, max_len=seq_len)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, sequence, segment_label):
    # Get positions tensor on the same device as sequence
      positions = self.position(sequence)

      # Ensure all tensors are on same device
      x = self.token(sequence) + positions + self.segment(segment_label)
      return self.dropout(x)

# Cell for attention layers
class MultiHeadedAttention(torch.nn.Module):
    def __init__(self, heads, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()

        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = torch.nn.Dropout(dropout)

        self.query = torch.nn.Linear(d_model, d_model)
        self.key = torch.nn.Linear(d_model, d_model)
        self.value = torch.nn.Linear(d_model, d_model)
        self.output_linear = torch.nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask):
        """
        query, key, value of shape: (batch_size, max_len, d_model)
        mask of shape: (batch_size, 1, 1, max_words)
        """
        # (batch_size, max_len, d_model)
        query = self.query(query)
        key = self.key(key)
        value = self.value(value)

        # (batch_size, max_len, d_model) --> (batch_size, max_len, h, d_k) --> (batch_size, h, max_len, d_k)
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        key = key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        value = value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)

        # (batch_size, h, max_len, d_k) matmul (batch_size, h, d_k, max_len) --> (batch_size, h, max_len, max_len)
        scores = torch.matmul(query, key.permute(0, 1, 3, 2)) / math.sqrt(query.size(-1))

        # fill 0 mask with super small number so it won't affect the softmax weight
        # (batch_size, h, max_len, max_len)
        scores = scores.masked_fill(mask == 0, -1e9)

        # (batch_size, h, max_len, max_len)
        # softmax to put attention weight for all non-pad tokens
        # max_len X max_len matrix of attention
        weights = F.softmax(scores, dim=-1)
        weights = self.dropout(weights)

        # (batch_size, h, max_len, max_len) matmul (batch_size, h, max_len, d_k) --> (batch_size, h, max_len, d_k)
        context = torch.matmul(weights, value)

        # (batch_size, h, max_len, d_k) --> (batch_size, max_len, h, d_k) --> (batch_size, max_len, d_model)
        context = context.permute(0, 2, 1, 3).contiguous().view(context.shape[0], -1, self.heads * self.d_k)

        # (batch_size, max_len, d_model)
        return self.output_linear(context)

class FeedForward(torch.nn.Module):
    "Implements FFN equation"

    def __init__(self, d_model, middle_dim=2048, dropout=0.1):
        super(FeedForward, self).__init__()

        self.fc1 = torch.nn.Linear(d_model, middle_dim)
        self.fc2 = torch.nn.Linear(middle_dim, d_model)
        self.dropout = torch.nn.Dropout(dropout)
        self.activation = torch.nn.GELU()

    def forward(self, x):
        out = self.activation(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out

class EncoderLayer(torch.nn.Module):
    def __init__(
        self,
        d_model=768,
        heads=12,
        feed_forward_hidden=768 * 4,
        dropout=0.1
        ):
        super(EncoderLayer, self).__init__()
        self.layernorm = torch.nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadedAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model, middle_dim=feed_forward_hidden)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, embeddings, mask):
        # embeddings: (batch_size, max_len, d_model)
        # encoder mask: (batch_size, 1, 1, max_len)
        # result: (batch_size, max_len, d_model)
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        # residual layer
        interacted = self.layernorm(interacted + embeddings)
        # bottleneck
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

# Cell for BERT model
class BERT(torch.nn.Module):
    """
    BERT model : Bidirectional Encoder Representations from Transformers.
    """

    def __init__(self, vocab_size, d_model=768, n_layers=12, heads=12, dropout=0.1):
        """
        :param vocab_size: vocab_size of total words
        :param hidden: BERT model hidden size
        :param n_layers: numbers of Transformer blocks(layers)
        :param attn_heads: number of attention heads
        :param dropout: dropout rate
        """

        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        self.heads = heads

        # paper noted they used 4*hidden_size for ff_network_hidden_size
        self.feed_forward_hidden = d_model * 4

        # embedding for BERT, sum of positional, segment, token embeddings
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=d_model)

        # multi-layers transformer blocks, deep network
        self.encoder_blocks = torch.nn.ModuleList(
            [EncoderLayer(d_model, heads, d_model * 4, dropout) for _ in range(n_layers)])

    def forward(self, x, segment_info):
        # attention masking for padded token
        # (batch_size, 1, seq_len, seq_len)
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)

        # embedding the indexed sequence to sequence of vectors
        x = self.embedding(x, segment_info)

        # running over multiple transformer blocks
        for encoder in self.encoder_blocks:
            x = encoder.forward(x, mask)
        return x

class NextSentencePrediction(torch.nn.Module):
    """
    2-class classification model : is_next, is_not_next
    """

    def __init__(self, hidden):
        """
        :param hidden: BERT model output size
        """
        super().__init__()
        self.linear = torch.nn.Linear(hidden, 2)
        self.softmax = torch.nn.LogSoftmax(dim=-1)

    def forward(self, x):
        # use only the first token which is the [CLS]
        return self.softmax(self.linear(x[:, 0]))

class MaskedLanguageModel(torch.nn.Module):
    """
    predicting origin token from masked input sequence
    n-class classification problem, n-class = vocab_size
    """

    def __init__(self, hidden, vocab_size):
        """
        :param hidden: output size of BERT model
        :param vocab_size: total vocab size
        """
        super().__init__()
        self.linear = torch.nn.Linear(hidden, vocab_size)
        self.softmax = torch.nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))

class BERTLM(torch.nn.Module):
    """
    BERT Language Model
    Next Sentence Prediction Model + Masked Language Model
    """

    def __init__(self, bert: BERT, vocab_size):
        """
        :param bert: BERT model which should be trained
        :param vocab_size: total vocab size for masked_lm
        """

        super().__init__()
        self.bert = bert
        self.next_sentence = NextSentencePrediction(self.bert.d_model)
        self.mask_lm = MaskedLanguageModel(self.bert.d_model, vocab_size)

    def forward(self, x, segment_label):
        x = self.bert(x, segment_label)
        return self.next_sentence(x), self.mask_lm(x)

# Training configuration
class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

class BERTTrainer:
    def __init__(
        self,
        model,
        train_dataloader,
        test_dataloader=None,
        lr=1e-4,
        weight_decay=0.01,
        betas=(0.9, 0.999),
        warmup_steps=10000,
        log_freq=10,
        device='cuda'
        ):

        self.device = device
        self.model = model
        self.train_data = train_dataloader
        self.test_data = test_dataloader

        # Setting the Adam optimizer with hyper-param
        self.optim = Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        self.optim_schedule = ScheduledOptim(
            self.optim, self.model.bert.d_model, n_warmup_steps=warmup_steps
            )

        # Using Negative Log Likelihood Loss function for predicting the masked_token
        self.criterion = torch.nn.NLLLoss(ignore_index=0)
        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))

    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def test(self, epoch):
        self.iteration(epoch, self.test_data, train=False)

    def iteration(self, epoch, data_loader, train=True):
        avg_loss = 0.0
        total_correct = 0
        total_element = 0

        mode = "train" if train else "test"

        # progress bar
        data_iter = tqdm.tqdm(
            enumerate(data_loader),
            desc="EP_%s:%d" % (mode, epoch),
            total=len(data_loader),
            bar_format="{l_bar}{r_bar}"
        )

        for i, data in data_iter:
            # 0. batch_data will be sent into the device(GPU or cpu)
            data = {key: value.to(self.device) for key, value in data.items()}

            # 1. forward the next_sentence_prediction and masked_lm model
            next_sent_output, mask_lm_output = self.model.forward(data["bert_input"], data["segment_label"])

            # 2-1. NLL(negative log likelihood) loss of is_next classification result
            next_loss = self.criterion(next_sent_output, data["is_next"])

            # 2-2. NLLLoss of predicting masked token word
            # transpose to (m, vocab_size, seq_len) vs (m, seq_len)
            mask_loss = self.criterion(mask_lm_output.transpose(1, 2), data["bert_label"])

            # 2-3. Adding next_loss and mask_loss
            loss = next_loss + mask_loss

            # 3. backward and optimization only in train
            if train:
                self.optim_schedule.zero_grad()
                loss.backward()
                self.optim_schedule.step_and_update_lr()

            # next sentence prediction accuracy
            correct = next_sent_output.argmax(dim=-1).eq(data["is_next"]).sum().item()
            avg_loss += loss.item()
            total_correct += correct
            total_element += data["is_next"].nelement()

            post_fix = {
                "epoch": epoch,
                "iter": i,
                "avg_loss": avg_loss / (i + 1),
                "avg_acc": total_correct / total_element * 100,
                "loss": loss.item()
            }

            if i % self.log_freq == 0:
                data_iter.write(str(post_fix))

            # Logging to WandB
            if i % 300 == 0 and train:
                metrics = {
                    "avg_loss": avg_loss / (i + 1),
                    "avg_acc": total_correct / total_element * 100,
                }

                wandb.log(metrics)

                # Save model checkpoint
                # save_model(
                #     self.model,
                #     self.optim,
                #     metrics,
                #     epoch,
                #     os.path.join('/content/drive/MyDrive/email_bert/checkpoints', f'bert_email_checkpoint_{epoch}_{i}.pth')
                # )
                print("Saved checkpoint model")

        print(
            f"EP{epoch}, {mode}: avg_loss={avg_loss / len(data_iter)}, total_acc={total_correct * 100.0 / total_element}"
        )

# Helper functions to save/load model
def save_model(model, optimizer, metrics, epoch, path):
    torch.save(
        {'model_state_dict': model.state_dict(),
         'optimizer_state_dict': optimizer.state_dict(),
         'metric': metrics,
         'epoch': epoch},
         path)

def load_model(model, optimizer=None, path='./checkpoint.pth'):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        optimizer = None
    epoch = checkpoint['epoch']
    metrics = checkpoint['metric']
    return model, optimizer, epoch, metrics

# Initialize WandB
import wandb
wandb.login()  # You'll need your API key
run = wandb.init(
    name="bert-email-pretraining",
    project="bert-email-project",
)

# Initialize model and trainer
print("Initializing model...")
bert_model = BERT(len(tokenizer.vocab))
bert_lm = BERTLM(bert_model, len(tokenizer.vocab))

# Use CPU if no GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
bert_lm.to(device)

bert_trainer = BERTTrainer(bert_lm, train_loader, device=device)

# Train for a few epochs
epochs = 5
for epoch in range(epochs):
    bert_trainer.train(epoch)

    # Save model after each epoch
    epoch_path = os.path.join('/content/drive/MyDrive/email_bert/checkpoints', f'bert_email_epoch_{epoch}.pth')
    save_model(bert_lm, bert_trainer.optim, {"epoch": epoch}, epoch, epoch_path)
    print(f"Saved model after epoch {epoch} to {epoch_path}")

# Save the final model
final_path = '/content/drive/MyDrive/email_bert/bert_email_final.pth'
save_model(bert_lm, bert_trainer.optim, {"final_epoch": epochs}, epochs, final_path)
print(f"Pre-training complete. Final model saved to {final_path}")

# Finish WandB run
wandb.finish()

# Additional info about the model location
print("\nModel and data files are saved to Google Drive in the following locations:")
print("- Dataset: /content/drive/MyDrive/email_bert/datasets/")
print("- Tokenizer: /content/drive/MyDrive/email_bert/tokenizer/")
print("- Checkpoints: /content/drive/MyDrive/email_bert/checkpoints/")
print("- Final model: /content/drive/MyDrive/email_bert/bert_email_final.pth")
print("\nThese files will persist even after your Colab session ends.")

Processing Enron emails...
Created 42439 email pairs
Sample pair: ['com> on 10/27/2000 09:54:20 AM To: "\'Rufino Doroteo\'" <rdoroteo@ect', "com> cc: Subject: FW: Overworked > -----Original Message----- > From: Rowland J (Janina) > Sent: Friday, October 27, 2000 9:46 AM > To: Marshall SS (Samantha); McLain CD (Carol); Waddilove P (Pamm); Loera > M (Maribel); Buentello SA (Sky); Torres J (Juanita); > 'pbrauer@kandplaw"]


100%|██████████| 84878/84878 [00:00<00:00, 633206.93it/s]


Created 9 text files for tokenizer training


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1964: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Sample tokenization: ['[CLS]', 'this', 'is', 'an', 'email', 'about', 'the', 'project', 'deadline', 'tomorrow', '[UNK]', '[SEP]']
Creating BERT dataset...
Batch Size torch.Size([32, 64])
Sample data item: {'bert_input': torch.Size([64]), 'bert_label': torch.Size([64]), 'segment_label': torch.Size([64]), 'is_next': torch.Size([])}


...

{'epoch': 4, 'iter': 1320, 'avg_loss': 6.544365548618267, 'avg_acc': 50.151400454201365, 'loss': 6.8060526847839355}


EP_train:4: 100%|| 1327/1327 [12:19<00:00,  1.79it/s]


EP4, train: avg_loss=6.543922231353668, total_acc=50.126063290840975
Saved model after epoch 4 to /content/drive/MyDrive/email_bert/checkpoints/bert_email_epoch_4.pth
Pre-training complete. Final model saved to /content/drive/MyDrive/email_bert/bert_email_final.pth


avg_acc,▃▄▄▄▄▃▄▄▄▄█▄▄▄▄▅▄▄▄▄▁▄▄▄▄
avg_loss,█▆▅▄▄▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂
avg_acc,50.07806
avg_loss,6.54628



Model and data files are saved to Google Drive in the following locations:
- Dataset: /content/drive/MyDrive/email_bert/datasets/
- Tokenizer: /content/drive/MyDrive/email_bert/tokenizer/
- Checkpoints: /content/drive/MyDrive/email_bert/checkpoints/
- Final model: /content/drive/MyDrive/email_bert/bert_email_final.pth

These files will persist even after your Colab session ends.


In [7]:
from google.colab import drive
drive.mount('/content/drive')
!ls -la /content/drive/MyDrive/email_bert/checkpoints/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 0
